<a href="https://colab.research.google.com/github/vkstar444/moving-object-detection/blob/main/moving_object_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **installs model**

In [ ]:
!nvidia-smi

In [ ]:
!pip install ultralytics

In [ ]:
! pip install opencv-python gradio


In [ ]:
import os
HOME = os.getcwd()
print(HOME)

In [ ]:
# Pip install method (recommended)

!pip install ultralytics==8.2.103 -q

from IPython import display
display.clear_output()

import ultralytics
ultralytics.checks()

## **import library's**

In [ ]:
#@title Imports and function definitions

# For running inference on the TF-Hub module.
import tensorflow as tf

import tensorflow_hub as hub

# For downloading the image.
import matplotlib.pyplot as plt
import tempfile
from six.moves.urllib.request import urlopen
from six import BytesIO

# For drawing onto the image.
import numpy as np
from PIL import Image
from PIL import ImageColor
from PIL import ImageDraw
from PIL import ImageFont
from PIL import ImageOps

# For measuring the inference time.
import time

# Print Tensorflow version
print(tf.__version__)

# Check available GPU devices.
print("The following GPU devices are available: %s" % tf.test.gpu_device_name())

In [ ]:
from ultralytics import YOLO

from IPython.display import display, Image

import cv2
import gradio as gr
import numpy as np
import matplotlib.pyplot as plt

## **Model**

In [ ]:


def vid_inf(vid_path):
    # Load the YOLO model (you can choose a specific model like 'yolov8n.pt')
    model = YOLO('yolov8n.pt')  # or 'yolov5s.pt', etc.

    # Create a VideoCapture object
    cap = cv2.VideoCapture(vid_path)

    # Get the video frames' width and height for proper saving of videos
    frame_width = int(cap.get(3))
    frame_height = int(cap.get(4))
    fps = int(cap.get(cv2.CAP_PROP_FPS))
    frame_size = (frame_width, frame_height)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    output_video = "output_recorded.mp4"

    # Create the VideoWriter object
    out = cv2.VideoWriter(output_video, fourcc, fps, frame_size)

    # Check if camera opened successfully
    if not cap.isOpened():
        print("Error opening video file")
        return None, None

    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if ret:
            # Perform object detection
            results = model(frame)

            # Draw bounding boxes and labels on the frame
            for result in results:
                boxes = result.boxes  # Get boxes
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0]  # Get box coordinates
                    conf = box.conf[0]  # Get confidence
                    cls = int(box.cls[0])  # Get class index
                    label = f'{model.names[cls]}: {conf:.2f}'  # Class name and confidence

                    # Draw rectangle and label on the frame
                    cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
                    cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            # Write the processed frame to the output video
            out.write(frame)

            # Convert frame for display
            frame_out_display = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            # Update the count every frame and yield every 12th frame
            if count % 12 == 0:
                yield frame_out_display, None

            count += 1

            # Press Q on keyboard to exit
            if cv2.waitKey(25) & 0xFF == ord('q'):
                break
        else:
            break

    # Release the video capture and writer object
    cap.release()
    out.release()
    cv2.destroyAllWindows()
    yield None, output_video

# Gradio interface
input_video = gr.Video(label="Input Video")
output_frames = gr.Image(label="Output Frames")
output_video_file = gr.Video(label="Output video")

app = gr.Interface(
    fn=vid_inf,
    inputs=[input_video],
    outputs=[output_frames, output_video_file],
    title="MotionScope with YOLO",
    description='A Gradio app for dynamic video analysis using YOLO object detection to identify and track moving objects in real-time.',
    allow_flagging="never",
    examples=[["sample/car.mp4"]],
)

app.queue().launch()